# build python exceptions from axe rules

i need better axe integration for testing with python systems. in the past i have inferred, or dynamically derived classes for the axe exceptions. the dynamic approach is wrong, our exceptions should be deterministic and closed work. this notebook derive the process for extracting axe rules.

## get the list of rules from `axe` in node

In [1]:
%%file get_axe_rules.mjs
import axe from "axe-core";

console.log(JSON.stringify(axe.getRules(), null, 2))

Overwriting get_axe_rules.mjs


In [2]:
rules = json.loads(subprocess.check_output(shlex.split("node get_axe_rules.mjs")))
DataFrame(rules)

,ruleId,description,help,helpUrl,tags,actIds
0,accesskeys,Ensures every accesskey attribute value is unique,accesskey attribute value should be unique,https://dequeuniversity.com/rules/axe/4.8/acce...,"[cat.keyboard, best-practice]",NaN
1,area-alt,Ensures <area> elements of image maps have alt...,Active <area> elements must have alternate text,https://dequeuniversity.com/rules/axe/4.8/area...,"[cat.text-alternatives, wcag2a, wcag244, wcag4...",[c487ae]
2,aria-allowed-attr,Ensures an element's role supports its ARIA at...,Elements must only use supported ARIA attributes,https://dequeuniversity.com/rules/axe/4.8/aria...,"[cat.aria, wcag2a, wcag412, EN-301-549, EN-9.4...",[5c01ea]
3,aria-allowed-role,Ensures role attribute has an appropriate valu...,ARIA role should be appropriate for the element,https://dequeuniversity.com/rules/axe/4.8/aria...,"[cat.aria, best-practice]",NaN
4,aria-braille-equivalent,Ensure aria-braillelabel and aria-brailleroled...,aria-braille attributes must have a non-braill...,https://dequeuniversity.com/rules/axe/4.8/aria...,"[cat.aria, wcag2a, wcag412, EN-301-549, EN-9.4...",NaN
...,...,...,...,...,...,...
98,td-has-header,Ensure that each non-empty data cell in a <tab...,Non-empty <td> elements in larger <table> must...,https://dequeuniversity.com/rules/axe/4.8/td-h...,"[cat.tables, experimental, wcag2a, wcag131, se...",NaN
99,td-headers-attr,Ensure that each cell in a table that uses the...,Table cells that use the headers attribute mus...,https://dequeuniversity.com/rules/axe/4.8/td-h...,"[cat.tables, wcag2a, wcag131, section508, sect...",[a25f45]
100,th-has-data-cells,Ensure that <th> elements and elements with ro...,Table headers in a data table must refer to da...,https://dequeuniversity.com/rules/axe/4.8/th-h...,"[cat.tables, wcag2a, wcag131, section508, sect...",[d0f69e]
101,valid-lang,Ensures lang attributes have valid values,lang attribute must have a valid value,https://dequeuniversity.com/rules/axe/4.8/vali...,"[cat.language, wcag2aa, wcag312, TTv5, TT11.b,...",[de46e4]


## using test frameworks as base classes

tags define testing frameworks. there is finite set of tags smaller than the number of rules.

In [3]:
tags = set()
for rule in rules:
    tags.update(rule.get("tags") or ())

In [4]:
def tag_to_class_name(tag):
    """transform a tag name into valid python"""
    if tag.startswith(("TT", "EN", "section508")):
        tag = tag.replace(*"._")
    if tag.startswith("cat"):
        tag = tag.partition(".")[2]
    return tag.replace(*"-_")


In [5]:
def tag_to_class(tag):
    """convert an axe tag to a python class"""

    return F"""class {tag_to_class_name(tag)}: ..."""

#### generate all the classes for the base tags

create classes for all the tags

In [6]:
body = io.StringIO()
had_cats = cats = False
for tag in sorted(tags):
    py = tag_to_class(tag)
    cats = tag.startswith("cat")
    if not had_cats and cats:
        body.write("class cat:")
        body.write("\n"*2)
    if cats:
        body.write(" "*4)    
    body.write(py)
    body.write("\n"*2)
    had_cats = cats

assert compile(body.getvalue(), "test", "exec"), "the python code is not valid"

## generate the base classes for each possible rule

In [7]:
def rule_to_class(ruleId, description, help, helpUrl=None, tags=None, **kwargs):
    """create python code from a rule payload"""
    base = "object"
    if tags:
        base = ", ".join(map(tag_to_class_name, tags))
    return F"""
class {ruleId_to_name(ruleId)}({base}):
    \"\"\"{description}
    
{help}

{helpUrl}\"\"\"
    
""".lstrip()
    # the helpurl will be clickable in a few contexts

In [8]:
def ruleId_to_name(ruleId):
    return ruleId.replace(*"-_")

In [9]:
body.write("\n"*2)

for rule in rules:
    body.write(rule_to_class(**rule))
    body.write("\n"*2)

In [13]:
assert compile(body.getvalue(), "", "exec")

In [11]:
print(body.getvalue())

class ACT: ...

class EN_301_549: ...

class EN_9_1_1_1: ...

class EN_9_1_2_1: ...

class EN_9_1_2_2: ...

class EN_9_1_3_1: ...

class EN_9_1_3_4: ...

class EN_9_1_3_5: ...

class EN_9_1_4_1: ...

class EN_9_1_4_12: ...

class EN_9_1_4_2: ...

class EN_9_1_4_3: ...

class EN_9_1_4_4: ...

class EN_9_2_1_1: ...

class EN_9_2_2_1: ...

class EN_9_2_2_2: ...

class EN_9_2_4_1: ...

class EN_9_2_4_2: ...

class EN_9_2_4_4: ...

class EN_9_2_5_3: ...

class EN_9_3_1_1: ...

class EN_9_3_1_2: ...

class EN_9_3_3_2: ...

class EN_9_4_1_2: ...

class TT11_a: ...

class TT11_b: ...

class TT12_a: ...

class TT12_d: ...

class TT13_a: ...

class TT13_c: ...

class TT14_b: ...

class TT17_a: ...

class TT2_a: ...

class TT2_b: ...

class TT4_a: ...

class TT5_c: ...

class TT6_a: ...

class TT7_a: ...

class TT7_b: ...

class TT8_a: ...

class TT9_a: ...

class TTv5: ...

class best_practice: ...

class cat:

    class aria: ...

    class color: ...

    class forms: ...

    class keyboard: 